In [23]:
import requests
import pandas as pd
import json
import numpy as np
import os
from config import API_KEY
api_key = os.environ.get('API_KEY', API_KEY)

HEADER = API_KEY

url_infla = 'https://api.estadisticasbcra.com/inflacion_mensual_oficial'
response_infla = requests.get(url_infla, headers=HEADER)
print(response_infla)
json_data_infla = response_infla.json()
df_infla = pd.json_normalize(json_data_infla)
df2_infla=df_infla
#agrego infla de enero manualmente
infla_enero = [18]
nueva_fecha = '2024-01-31'

# Usar el método loc para agregar la nueva fila al final
df2_infla.loc[len(df2_infla)] = [nueva_fecha] + infla_enero

df2_infla


<Response [200]>


,d,v
0,1990-01-31,79.200000000000003
1,1990-02-28,61.600000000000001
2,1990-03-31,95.500000000000000
3,1990-04-30,11.400000000000000
4,1990-05-31,13.600000000000000
...,...,...
404,2023-09-30,12.699999999999999
405,2023-10-31,8.300000000000001
406,2023-11-30,12.800000000000001
407,2023-12-31,25.500000000000000


In [8]:

#Acá creamos un diccionario/lista con la inflacion diaria (promedio geome) para cada mes.
infla_diaria = []
fecha = []
for indice in range(len(df2_infla)):
    dias_por_mes = df2_infla.at[df2_infla.index[indice], 'd'].split('-')[2]
    dias_por_mes = int(dias_por_mes)
    dias_por_mes = np.array(dias_por_mes)
    fecha_f = df2_infla.at[df2_infla.index[indice], 'd']
    dif_ipc_mensual = df2_infla.at[df2_infla.index[indice], 'v']
    infla_dia = (1 + dif_ipc_mensual / 100) ** (1/dias_por_mes)
    infla_diaria.append(infla_dia)
    fecha.append(fecha_f)

pd.set_option('display.float_format', '{:.15f}'.format) #aumento el nro de decimales disponibles en el data frame
df_infla_diaria = pd.DataFrame()
df_infla_diaria['Infla_Diaria'] = infla_diaria


In [9]:
df_fecha = pd.DataFrame(fecha, columns=['date'])

# Convertir la columna 'Fecha' de str a datetime
df_fecha = pd.to_datetime(df_fecha['date'])

df_merge_infla = pd.merge(df_fecha, df_infla_diaria, left_index=True, right_index=True, how='inner')
df_merge_infla

,date,Infla_Diaria
0,1990-01-31,1.018995330134033
1,1990-02-28,1.017288966467890
2,1990-03-31,1.021861015743875
3,1990-04-30,1.003605054015243
4,1990-05-31,1.004121804277895
...,...,...
404,2023-09-30,1.003993259734605
405,2023-10-31,1.002575406418648
406,2023-11-30,1.004022942163776
407,2023-12-31,1.007353861758708


In [10]:
date_ranges = pd.date_range(start='1990-01-01', end=df_merge_infla['date'].max(), freq='D')

# Crear un nuevo dataframe con el rango de fechas
df_expanded = pd.DataFrame({'date': date_ranges})

# Unir los dataframes original y expandido usando left join
df_merged = pd.merge(df_expanded, df_merge_infla, on='date', how='left')

# Rellenar los valores de inflación diaria con el método de propagación hacia adelante (forward fill)
df_merged['Infla_Diaria'] = df_merged['Infla_Diaria'].bfill()

# Mostrar el nuevo dataframe resultante
df_merged

,date,Infla_Diaria
0,1990-01-01,1.018995330134033
1,1990-01-02,1.018995330134033
2,1990-01-03,1.018995330134033
3,1990-01-04,1.018995330134033
4,1990-01-05,1.018995330134033
...,...,...
12444,2024-01-27,1.005353454232947
12445,2024-01-28,1.005353454232947
12446,2024-01-29,1.005353454232947
12447,2024-01-30,1.005353454232947


In [11]:
pd.reset_option('display.max_rows')
df_merged['Factor_Actualizacion'] = df_merged['Infla_Diaria'][::-1].cumprod()[::-1]
df_merged

,date,Infla_Diaria,Factor_Actualizacion
0,1990-01-01,1.018995330134033,9772.229412358325135
1,1990-01-02,1.018995330134033,9590.062999673351442
2,1990-01-03,1.018995330134033,9411.292393667716169
3,1990-01-04,1.018995330134033,9235.854292315358180
4,1990-01-05,1.018995330134033,9063.686573618077091
...,...,...,...
12444,2024-01-27,1.005353454232947,1.027055404269932
12445,2024-01-28,1.005353454232947,1.021586388295192
12446,2024-01-29,1.005353454232947,1.016146494542688
12447,2024-01-30,1.005353454232947,1.010735567938119


In [15]:
import plotly.express as px
fig_infla = px.line(df_merged, x='date', y='Infla_Diaria', title='infla diaria 1990-2024', hover_data='Factor_Actualizacion')
fig_infla

In [21]:
#df_merged.to_csv('factor_actualizacion.csv', index=False)
